In [1]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem.rdMolDescriptors import CalcNumHBA
from rdkit.Chem.rdMolDescriptors import CalcNumHBD

import numpy as np
import pandas as pd

import os

## bring DB --> primary processing

In [2]:
# 승연님(인턴)) 수집 데이터

folder_path = "/home/ufslab223/Shared/KSW/torch_geometric/TLC_RF/DB/collecting/intern_sy/Rf data1_raw.csv"

sy_df = pd.read_csv(folder_path)
sy_df.drop(columns=['Unnamed: 0'], inplace = True)
sy_df



,compound name,Smiles,eluent1,eluent2,ratio,Rf,title,doi,publish
0,17b,C(C1=CC=CC=C1)N1C[C@H]2[C@@](C1)(CCC2)P(C)(C)=O,dichloromethane,methanol,95/5,0.26,P(O)Me2–Alkenes: From Synthesis to Applications,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,"2025,27,23"
1,23b,COC=1C=C(C=CC1)C(=C)P(C)(C)=O,dichloromethane,methanol,95/5,0.21,P(O)Me2–Alkenes: From Synthesis to Applications,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,"2025,27,23"
2,(7b),CP(=O)(C)C1=C[C@H]2CC[C@@H](C1)N2C(=O)[O-],dichloromethane,methanol,95/5,0.18,P(O)Me2–Alkenes: From Synthesis to Applications,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,"2025,27,23"
3,"(2,5-Dihydrofuran-3-yl)dimethylphosphine oxide...",O1CC(=CC1)P(C)(C)=O,dichloromethane,methanol,95/5,0.23,P(O)Me2–Alkenes: From Synthesis to Applications,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,"2025,27,23"
4,19b,O1CC(=CC1)P(C)(C)=O,dichloromethane,methanol,95/5,0.26,P(O)Me2–Alkenes: From Synthesis to Applications,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,"2025,27,23"
...,...,...,...,...,...,...,...,...,...
21401,cis-3ad,C1(=CC=CC2=CC=CC=C12)C=1OC(=C2C1C(OCC2C=C)C2=C...,PE,EtOAc,40/1,0.15,Highly Regio- and Diastereoselective Synthesis...,https://pubs.acs.org/doi/10.1021/acs.orglett.4...,"2024,26,10"
21402,trans-3ad,C1(=CC=CC2=CC=CC=C12)C=1OC(=C2C1C(OCC2C=C)C2=C...,PE,EtOAc,40/1,0.19,Highly Regio- and Diastereoselective Synthesis...,https://pubs.acs.org/doi/10.1021/acs.orglett.4...,"2024,26,10"
21403,3ak,C1(=CC=CC=C1)C=1OC(=C2[C@@H](OC[C@@H](C21)C=C)...,PE,EtOAc,5/95,0.3,Highly Regio- and Diastereoselective Synthesis...,https://pubs.acs.org/doi/10.1021/acs.orglett.4...,"2024,26,10"
21404,3ap,C1(=CC=CC=C1)C=1OC(=C2C1[C@@H](OC[C@@H]2C=C)C2...,PE,EtOAc,20/1,0.13,Highly Regio- and Diastereoselective Synthesis...,https://pubs.acs.org/doi/10.1021/acs.orglett.4...,"2024,26,10"


In [3]:
# 데이터 통일 및 결측치 제거
def isit_string_of_float(rf):
    try:
        float(rf)
        return True
    except:
        return False



sy_df.drop(columns=['compound name', 'publish'], inplace = True)
sy_df.dropna(subset=['Smiles', 'Rf'], inplace = True)    #Smiles nan 제거,  Rf nan 제거
sy_df.dropna(subset = ['eluent1', 'eluent2'], how = 'all', inplace = True) #eluent1,2 모두 없는거 제거
mask = sy_df['Rf'].apply(isit_string_of_float) # 가끔 Rf 가 문자열로 보관되는데, float으로 변환 가능하지만, 0.4 - 0.45 이런건 안되니 걍 지우자
sy_df = sy_df[mask]

# ratio 칼럼의 모두 string 으로 변환
# 밑에서 가공하려고
sy_df['ratio'] = sy_df['ratio'].astype(str)

# eluent 정체불명 제거
sy_df = sy_df[~((sy_df[['eluent1', 'eluent2']] == '1:1 DCM:Et2O').any(axis=1) | (sy_df[['eluent1', 'eluent2']] == 'Et2O/DCM').any(axis=1)| (sy_df[['eluent1', 'eluent2']] == 'ethyl').any(axis=1))]

sy_df.columns = ['Smiles', 'elu1', 'elu2', 'ratio', 'Rf', 'title', 'doi']

print(f'raw개수: {len(sy_df)}')


raw개수: 21274


/tmp/ipykernel_3879093/2031940554.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sy_df['ratio'] = sy_df['ratio'].astype(str)


In [5]:
# path='/home/ufslab223/Shared/ains/GPT/data_extracted_gpt'
# for file in os.listdir(path):
#     if os.path.isfile(os.path.join(path, file)):
#         f = pd.read_csv(os.path.join(path, file))
#         mask = ((f['eluent1']=="EE") | (f['eluent2']=="EE"))
#         ff = f[mask]
#         if (len(ff)>0):
#             a=ff
#             print (file)



In [4]:
##### eluent를 smiles로 변경하기 전에, elu 이름 표준화 및, smiles, 정보들과 매칭 시스템 #####



#필요 없는 칼럼 지우고 eluent 이름 통일한 후에 통일한 이름 바탕으로, smiles 로 변환하기
elu_list1 = list(sy_df['elu1'].drop_duplicates())
elu_list2 = list(sy_df['elu2'].drop_duplicates())
elu_list = list(pd.Series(elu_list1 + elu_list2).drop_duplicates()) # set 대신 이렇게 써야 요소 순서가 고정됨(등장 순서)
elu_list

# 처음 보는 eluent (elu_to_key_df에 없는) 추가되면 대응되는 key 지정해주기
elu_to_key_df = pd.read_csv('elu_to_key_df.csv') 
elu_df = pd.read_csv('eluent_standard.csv')
new_eluent=[]
for elu in elu_list:
    if elu in list(elu_to_key_df['elu_raw']):
        continue
    else:
        new_eluent.append(elu)

# 하나씩 대응되는 표준 key 지정
# eluent_standard_name.csv 에 각 number key와 eluent 표준명을 기재해놓음.
# 만약 여기에 없는 새로운 eluent가 추가되면 number key 하나 추가하여 갱신하면 됨.
elu_key=[]
for elu in new_eluent:
    elu_df = pd.read_csv('eluent_standard.csv') # for 문 내에서 업데이트 된 거 새로 불러오기
    key = int(input(f'{elu} 의 key 지정 --> '))
    elu_key.append(key)
    # 만약 추가되는 eluent가 이름만 새로운게 아니라 새로운 화학종이면 elu_standard에서 추가해놔야함
    if key > len(elu_df):
        std_name = input(f'{key}와 매칭될 표준 이름 입력: ')
        print(key, std_name)
        smiles = input(f'{key}와 매칭될 smiles 입력 (자동 canonicalize 해줄것임): ')
        if smiles == '??':
            cano_smiles = smiles # 논문 작성자가 무슨 용매를 의미한건지 명확하지 않을때 임시로 채워넣는것
        else:
            cano_smiles = Chem.MolToSmiles((Chem.MolFromSmiles(smiles)), canonical = True)
        density = input(f'{key}와 매칭될 밀도 입력: ')

        extra_eleunt_inform = pd.DataFrame({'key': [key],'std_name': [std_name],'567_Smiles': [cano_smiles],'hex_Smiles': [cano_smiles], 'density': [density]})
        elu_df = pd.concat([elu_df, extra_eleunt_inform], axis=0)
        elu_df.to_csv('eluent_standard.csv', index=False) # elu_standard 업데이트 저장


extra_elu2key_df = pd.DataFrame({'elu_raw': new_eluent, 'elu_key': elu_key})
elu_to_key_df = pd.concat([elu_to_key_df, extra_elu2key_df], axis=0)
elu_to_key_df.to_csv('elu_to_key_df.csv', index=False) # elu to key 업데이트 저장

In [5]:
# eluent --> key --> smiles 변환

elu_to_key_df = pd.read_csv('elu_to_key_df.csv') 
elu_to_key_converter = dict(zip(elu_to_key_df['elu_raw'], elu_to_key_df['elu_key']))

elu_df = pd.read_csv('eluent_standard.csv')
key_to_567smiles_converter = dict(zip(elu_df['key'], elu_df['567_Smiles']))
key_to_smiles_converter = dict(zip(elu_df['key'], elu_df['hex_Smiles']))

elu1_list_org = sy_df['elu1']
elu2_list_org = sy_df['elu2']

# -----------PE --> hex 버전 ---------------

sy_df['elu1'] = [key_to_smiles_converter[elu_to_key_converter[name]] for name in elu1_list_org]
sy_df['elu2']  = [key_to_smiles_converter[elu_to_key_converter[name]] for name in elu2_list_org]

# ratio 칼럼 정리
sy_df['ratio'] = [rat.replace('/',':',1) if '/' in rat else rat for rat in sy_df['ratio']]

ratio1 = [float(rat.split(':')[0]) / (float(rat.split(':')[0]) + float(rat.split(':')[1]))
          if isinstance(rat, str) and  ':' in rat else float(1)  for rat in sy_df['ratio']]
ratio2 = [float(rat.split(':')[1]
) / (float(rat.split(':')[0]) + float(rat.split(':')[1]))
          if isinstance(rat, str) and ':' in rat else float(0)  for rat in sy_df['ratio']]
sy_df['ratio1'] = ratio1
sy_df['ratio2'] = ratio2
sy_df.drop(columns = ['ratio'], inplace=True)
sy_df = sy_df[['Smiles', 'elu1', 'elu2', 'ratio1', 'ratio2', 'Rf', 'title', 'doi']]

sy_df['ratio1'] = sy_df['ratio1'].apply(lambda x: float(x) if not isinstance(x, float) else x)
sy_df['ratio2'] = sy_df['ratio2'].apply(lambda x: float(x) if not isinstance(x, float) else x)
sy_df['Rf'] = sy_df['Rf'].apply(lambda x: float(x) if not isinstance(x, float) else x)

#----------PE --> 567버전 ------------------
from copy import deepcopy
sy_df_567 = deepcopy(sy_df)
sy_df_567['elu1'] = [key_to_567smiles_converter[elu_to_key_converter[name]] for name in elu1_list_org]
sy_df_567['elu2']  = [key_to_567smiles_converter[elu_to_key_converter[name]] for name in elu2_list_org]

In [6]:
sy_df_567

,Smiles,elu1,elu2,ratio1,ratio2,Rf,title,doi
0,C(C1=CC=CC=C1)N1C[C@H]2[C@@](C1)(CCC2)P(C)(C)=O,ClCCl,CO,0.950000,0.050000,0.26,P(O)Me2–Alkenes: From Synthesis to Applications,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
1,COC=1C=C(C=CC1)C(=C)P(C)(C)=O,ClCCl,CO,0.950000,0.050000,0.21,P(O)Me2–Alkenes: From Synthesis to Applications,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
2,CP(=O)(C)C1=C[C@H]2CC[C@@H](C1)N2C(=O)[O-],ClCCl,CO,0.950000,0.050000,0.18,P(O)Me2–Alkenes: From Synthesis to Applications,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
3,O1CC(=CC1)P(C)(C)=O,ClCCl,CO,0.950000,0.050000,0.23,P(O)Me2–Alkenes: From Synthesis to Applications,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
4,O1CC(=CC1)P(C)(C)=O,ClCCl,CO,0.950000,0.050000,0.26,P(O)Me2–Alkenes: From Synthesis to Applications,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
...,...,...,...,...,...,...,...,...
21401,C1(=CC=CC2=CC=CC=C12)C=1OC(=C2C1C(OCC2C=C)C2=C...,CCCCC.CCCCCC.CCCCCCC,CCOC(C)=O,0.975610,0.024390,0.15,Highly Regio- and Diastereoselective Synthesis...,https://pubs.acs.org/doi/10.1021/acs.orglett.4...
21402,C1(=CC=CC2=CC=CC=C12)C=1OC(=C2C1C(OCC2C=C)C2=C...,CCCCC.CCCCCC.CCCCCCC,CCOC(C)=O,0.975610,0.024390,0.19,Highly Regio- and Diastereoselective Synthesis...,https://pubs.acs.org/doi/10.1021/acs.orglett.4...
21403,C1(=CC=CC=C1)C=1OC(=C2[C@@H](OC[C@@H](C21)C=C)...,CCCCC.CCCCCC.CCCCCCC,CCOC(C)=O,0.050000,0.950000,0.30,Highly Regio- and Diastereoselective Synthesis...,https://pubs.acs.org/doi/10.1021/acs.orglett.4...
21404,C1(=CC=CC=C1)C=1OC(=C2C1[C@@H](OC[C@@H]2C=C)C2...,CCCCC.CCCCCC.CCCCCCC,CCOC(C)=O,0.952381,0.047619,0.13,Highly Regio- and Diastereoselective Synthesis...,https://pubs.acs.org/doi/10.1021/acs.orglett.4...


In [7]:
import sys
sys.path.append('../../../')
from functions import delete_weird_mol_and_None_except_Nan, delete_metal_mol_except_Nan


# ---------------PE_hex 버전

print('1')
sy_df = delete_weird_mol_and_None_except_Nan(sy_df, 'Smiles')
print('2')
sy_df = delete_metal_mol_except_Nan(sy_df, 'Smiles')
print('3')
sy_df = delete_weird_mol_and_None_except_Nan(sy_df, 'elu1')
print('4')
sy_df = delete_metal_mol_except_Nan(sy_df, 'elu1')
print('5')
sy_df = delete_weird_mol_and_None_except_Nan(sy_df, 'elu2')
print('6')
sy_df = delete_metal_mol_except_Nan(sy_df, 'elu2')
print('7')


# ---------------PE_567 버전

print('1')
sy_df_567 = delete_weird_mol_and_None_except_Nan(sy_df_567, 'Smiles')
print('2')
sy_df_567 = delete_metal_mol_except_Nan(sy_df_567, 'Smiles')
print('3')
sy_df_567 = delete_weird_mol_and_None_except_Nan(sy_df_567, 'elu1')
print('4')
sy_df_567 = delete_metal_mol_except_Nan(sy_df_567, 'elu1')
print('5')
sy_df_567 = delete_weird_mol_and_None_except_Nan(sy_df_567, 'elu2')
print('6')
sy_df_567 = delete_metal_mol_except_Nan(sy_df_567, 'elu2')
print('7')

1


[16:11:23] Explicit valence for atom # 17 C, 5, is greater than permitted


4660번째 분자 mol=None (smiles가 아닌 값)


[16:11:25] Explicit valence for atom # 11 N, 4, is greater than permitted


13882번째 분자 mol=None (smiles가 아닌 값)


...

4660번째 분자 mol=None (smiles가 아닌 값)


[16:11:40] Explicit valence for atom # 11 N, 4, is greater than permitted


13882번째 분자 mol=None (smiles가 아닌 값)


[16:11:41] Explicit valence for atom # 13 N, 4, is greater than permitted
[16:11:41] Explicit valence for atom # 11 N, 4, is greater than permitted


20808번째 분자 mol=None (smiles가 아닌 값)
20810번째 분자 mol=None (smiles가 아닌 값)
2
117번째 분자 금속원소 포함됨
584번째 분자 금속원소 포함됨
3449번째 분자 금속원소 포함됨
4492번째 분자 금속원소 포함됨
4603번째 분자 금속원소 포함됨
4604번째 분자 금속원소 포함됨
4605번째 분자 금속원소 포함됨
5141번째 분자 금속원소 포함됨
5977번째 분자 금속원소 포함됨
5978번째 분자 금속원소 포함됨
5987번째 분자 금속원소 포함됨
5989번째 분자 금속원소 포함됨
5996번째 분자 금속원소 포함됨
5997번째 분자 금속원소 포함됨
6006번째 분자 금속원소 포함됨
6008번째 분자 금속원소 포함됨
6120번째 분자 금속원소 포함됨
6121번째 분자 금속원소 포함됨
6122번째 분자 금속원소 포함됨
8939번째 분자 금속원소 포함됨
3
4
5
6
7


In [8]:
#Smiles canonicalize

#-------------PE hex 버전 -------------

sy_df = sy_df[(0<sy_df['Rf']) & (sy_df['Rf']<=1)]

# ratio1 > ratio2 되도록 정렬 --> 중복 데이터 판별시 필요
switch_ratio_mask = sy_df['ratio1']-sy_df['ratio2']<0

sy_df.loc[switch_ratio_mask, ['ratio1','ratio2']] = sy_df.loc[switch_ratio_mask, ['ratio2', 'ratio1']].values
sy_df.loc[switch_ratio_mask, ['elu1','elu2']] = sy_df.loc[switch_ratio_mask, ['elu2', 'elu1']].values

# compound smiles canonicalize 
sy_df.loc[:,'Smiles'] = [Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True, canonical=True) for smiles in sy_df['Smiles']]

sy_df = sy_df.reset_index(drop=True)


#-------------PE 567 버전 -------------

sy_df_567 = sy_df_567[(0<sy_df_567['Rf']) & (sy_df_567['Rf']<=1)]

# ratio1 > ratio2 되도록 정렬 --> 중복 데이터 판별시 필요
switch_ratio_mask = sy_df_567['ratio1']-sy_df_567['ratio2']<0

sy_df_567.loc[switch_ratio_mask, ['ratio1','ratio2']] = sy_df_567.loc[switch_ratio_mask, ['ratio2', 'ratio1']].values
sy_df_567.loc[switch_ratio_mask, ['elu1','elu2']] = sy_df_567.loc[switch_ratio_mask, ['elu2', 'elu1']].values

# compound smiles canonicalize 
sy_df_567.loc[:,'Smiles'] = [Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True, canonical=True) for smiles in sy_df_567['Smiles']]

sy_df_567 = sy_df_567.reset_index(drop=True)


### elu1,elu2 도 canonlicalize 완료!

In [9]:
sy_df['elu1'] = [Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True) if pd.notna(s) else np.nan for s in sy_df['elu1']]
sy_df['elu2'] = [Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True) if pd.notna(s) else np.nan for s in sy_df['elu2']]

In [10]:
sy_df_567['elu1'] = [Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True) if pd.notna(s) else np.nan for s in sy_df_567['elu1']]
sy_df_567['elu2'] = [Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True) if pd.notna(s) else np.nan for s in sy_df_567['elu2']]

### Rf_type 추가:
#### 해당 논문에 정밀한 단위 Rf 있으면 정밀한 논문으로 취급
#### 0.05로 나누었을때 나머지 0 보다 크면 인정해줌

In [11]:
def set_Rf_type(df, not_005_ratio=0.5):
    from decimal import Decimal

    C_Rf_paper_list=[]
    for paper, paper_data in df.groupby(['title'], as_index=True):
        is_it_precise = np.array([Decimal(str(rf)) for rf in paper_data['Rf']]) % Decimal('0.05')
        # if (is_it_precise > 0).any():
        if len(is_it_precise[is_it_precise>0]) > len(is_it_precise)*not_005_ratio:
            C_Rf_paper_list.append(paper[0])

    df['Rf_type'] = ['C' if (title in C_Rf_paper_list) else 'Q' for title in df['title']]
    return df

In [12]:
sy_df_final = set_Rf_type(sy_df, 0.7)
sy_df_567_final = set_Rf_type(sy_df_567, 0.7)

In [27]:
len(sy_df_final[sy_df_final['Rf_type']=='C'])

5656

In [13]:
sy_df_final.to_csv('../collecting/primary_processed_PE_hex/intern_sy_PE_hex.csv', index=False)
sy_df_567_final.to_csv('../collecting/primary_processed_PE_567/intern_sy_PE_567.csv', index=False)

In [18]:
LLM_df_567

,Smiles,elu1,elu2,ratio1,ratio2,Rf,title,doi
0,COc1ccc(C#Cc2ccccc2C(=O)/C=C/c2ccc(I)cc2)cc1,CCCCCC,CCOC(C)=O,0.995,0.005,0.20,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
1,O=C1C(c2ccccc2)=C(c2ccccc2)c2ccccc21,CCCCCC,CCOC(C)=O,0.950,0.050,0.50,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
2,COc1ccc(C2=C(c3ccc(OC)cc3)c3ccccc3C2=O)cc1,CCCCCC,CCOC(C)=O,0.950,0.050,0.40,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
3,O=C1C(c2ccc(F)cc2)=C(c2ccc(F)cc2)c2ccccc21,CCCCCC,CCOC(C)=O,0.950,0.050,0.40,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
4,O=C1C(c2ccc(C(F)(F)F)cc2)=C(c2ccc(C(F)(F)F)cc2...,CCCCCC,CCOC(C)=O,0.950,0.050,0.40,Phosphine-Based Porous Organic Polymer-Stabili...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...
...,...,...,...,...,...,...,...,...
15298,CC(=O)OC1=C2CC(c3ccccc3)(C3CC3)C2CCC1,CCCCCC,CCOC(C)=O,1.000,0.000,0.50,"2 + 2 Trapping of Acyloxy-1,2-cyclohexadienes ...",https://pubs.acs.org/doi/10.1021/acs.orglett.2...
15299,CC(C)(C)C(=O)OC1=C2CC(C)(c3ccccc3)C2CCC1,CCCCCC,CCOC(C)=O,1.000,0.000,0.42,"2 + 2 Trapping of Acyloxy-1,2-cyclohexadienes ...",https://pubs.acs.org/doi/10.1021/acs.orglett.2...
15300,CCOC(=O)C1CC2=C(OC(=O)c3ccccc3)CCCC21,CCCCCC,CCOC(C)=O,1.000,0.000,0.29,"2 + 2 Trapping of Acyloxy-1,2-cyclohexadienes ...",https://pubs.acs.org/doi/10.1021/acs.orglett.2...
15301,COC(=O)C1(C)CC2=C(OC(C)=O)CCCC21,CCCCCC,CCOC(C)=O,1.000,0.000,0.44,"2 + 2 Trapping of Acyloxy-1,2-cyclohexadienes ...",https://pubs.acs.org/doi/10.1021/acs.orglett.2...
